In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import gmsh

<center><img src="Diseño Arco Recurvo.jpeg"></center>

In [2]:
gmsh.initialize()

In [3]:
gmsh.model.add('Medio Arco Recurvo')

In [4]:
lc = 1
E = 32e6 #Pa???????
#Defino los puntos iniciales del modelo
# p1 = gmsh.model.geo.addPoint(0, 0, 0, lc) 
p2 = gmsh.model.geo.addPoint(16, 0, 0, lc) 
p3 = gmsh.model.geo.addPoint(14, 10, 0, lc) 
p4 = gmsh.model.geo.addPoint(0, 78, 0, lc) 
p5 = gmsh.model.geo.addPoint(0, 80, 0, lc)
p6 = gmsh.model.geo.addPoint(2, 80, 0, lc)
p7 = gmsh.model.geo.addPoint(2, 78, 0, lc) 
p8 = gmsh.model.geo.addPoint(16, 10, 0, lc)
p9 = gmsh.model.geo.addPoint(18, 0, 0, lc) # optimizar estos puntos para optimizar 
p10 = gmsh.model.geo.addPoint(17, 5, 0, lc)
p11 = gmsh.model.geo.addPoint(19, 5, 0, lc)
#Defino las curvas que conformarán el contorno:
l1 = gmsh.model.geo.add_spline([p2,p10,p3])
l2 = gmsh.model.geo.addLine(p3, p4)
l3 = gmsh.model.geo.addLine(p4, p5)
l4 = gmsh.model.geo.addLine(p5, p6)
l5 = gmsh.model.geo.addLine(p6, p7)
l6 = gmsh.model.geo.addLine(p7, p8)
l7 = gmsh.model.geo.add_spline([p9,p11,p8])
l8 = gmsh.model.geo.addLine(p9, p2)
l9 = gmsh.model.geo.addLine(p3, p8)
l10 = gmsh.model.geo.addLine(p3, p8)
l11 = gmsh.model.geo.addLine(p4, p7) 
#Defino los contornos:
C_Traccionado= gmsh.model.geo.addCurveLoop([l3,l4,l5,l11],reorient=True)
C_Pala= gmsh.model.geo.addCurveLoop([l2,l11,l6,l10],reorient=True)
C_Grap= gmsh.model.geo.addCurveLoop([l1,l9,l7,l8],reorient=True)
#Defino las superficies:
S_Traccionado = gmsh.model.geo.addPlaneSurface([C_Traccionado])
S_Pala = gmsh.model.geo.addPlaneSurface([C_Pala])
S_Grap = gmsh.model.geo.addPlaneSurface([C_Grap])
gmsh.model.geo.synchronize()
#Extrudo el Volumen,para eso debo definir el PG y la Entitie de la superficie:
#gmsh.fltk.run()
V_Traccionado = gmsh.model.geo.extrude([(2,S_Traccionado)],0,0,3)
V_Pala = gmsh.model.geo.extrude([(2,S_Pala)],0,0,3)
V_Grap = gmsh.model.geo.extrude([(2,S_Grap)],0,0,3)
gmsh.model.geo.removeAllDuplicates()
gmsh.model.geo.synchronize()
gmsh.option.setNumber("Mesh.MeshSizeMin", 1.2)
gmsh.model.mesh.generate(dim=3)
gmsh.fltk.run()

In [5]:
NodeInfo = gmsh.model.mesh.get_nodes() #saco un objeto con todos los nodos. array uno etiquetas que asigna a los nodos, empieza con 1 y no con 0.
NN = NodeInfo[0].shape[0] #numero de nodos es la cantidad de elementos uqe tengo en el primer array
NN_Tags = NodeInfo[0]-1 #Los Tags de los nodos los pongo en numeración python.
MN = NodeInfo[1].reshape(NN,3)
GLXN=3
E_Tags, MC_Flatten = gmsh.model.mesh.get_elements_by_type(4) #dame los tags (numero de elementos) del tipo 4 (tetrahedros) la MC flatten.
NNXE=4
NE=E_Tags.shape[0]
MC=MC_Flatten.reshape(NE,4)
MC=MC-np.ones(MC.shape)#Lo dejo en numeración python

In [6]:
#Busco los tags de la superficie traccionada y del volumen del grap para armar las s.
Tag_S_Traccionada = [20] #la busque en el gmsh
Tag_V_Grap = [tag[1] for tag in V_Grap if tag[0]==3]
#Armo los Physical Groups
Traccionado_PG = gmsh.model.addPhysicalGroup(2, Tag_S_Traccionada) #Tracciono la superficie interna, donde estaría sujetada la cuerda.
Grap_PG = gmsh.model.addPhysicalGroup(3, Tag_V_Grap)

In [7]:
#Busco los tags del Grap, armo el s y el Us. El grap no se puede mover en ninguna de las 3 direcciones
Tags_Nodos_Grap, CoordFlatten_Nodos_Grap = gmsh.model.mesh.getNodesForPhysicalGroup(3, Grap_PG)
s=np.array([Tags_Nodos_Grap*GLXN,Tags_Nodos_Grap*GLXN+1,Tags_Nodos_Grap*GLXN+2]).flatten()
Us=np.zeros_like(s)
Tags_Nodos_Traccionados, CoordFlatten_Nodos_Traccionados = gmsh.model.mesh.getNodesForPhysicalGroup(2, Traccionado_PG)
r=np.array([Tags_Nodos_Traccionados*GLXN+1,Tags_Nodos_Traccionados*GLXN+2])#No estoy segura. 

In [8]:
#poner esquema arco, mostrando angulo, para mi apertura, mi arco es de 30 lbs tensado bla bla bla explicar porque elijo.

In [9]:
CoordFlatten_Nodos_Traccionados.reshape(len(Tags_Nodos_Traccionados),3)

array([[ 0.        , 78.        ,  0.        ],
       [ 0.        , 80.        ,  0.        ],
       [ 0.        , 78.        ,  3.        ],
       [ 0.        , 80.        ,  3.        ],
       [ 0.        , 79.        ,  0.        ],
       [ 0.        , 79.        ,  3.        ],
       [ 0.        , 78.        ,  1.        ],
       [ 0.        , 78.        ,  2.        ],
       [ 0.        , 80.        ,  1.        ],
       [ 0.        , 80.        ,  2.        ],
       [ 0.        , 78.90844789,  1.5       ],
       [ 0.        , 79.23969458,  2.28666398],
       [ 0.        , 79.23969458,  0.71333602],
       [ 0.        , 78.60774666,  2.36812459],
       [ 0.        , 78.60774665,  0.63187541]])

In [10]:
Tags_Nodos_Traccionados

array([  3,   4,  11,  12,  87, 159, 163, 164, 165, 166, 385, 386, 387,
       388, 389], dtype=uint64)